
# 📓 SmartWaste — Aprendizaje Supervisado de Rutas (Notebook Explicado)

Este notebook replica y **explica paso a paso** el flujo de tu módulo `ml_supervisado.py`, pero en formato interactivo de Jupyter Notebook.  
Al final tendrás:
- Un modelo entrenado (`public/modelos/modelo_supervisado.pkl`)
- Gráficas guardadas en `public/graficas/`
- Métricas de evaluación (accuracy y reporte de clasificación)

> **Objetivo:** clasificar rutas como **Eficientes (1)** o **No eficientes (0)** usando datos de bitácoras de rutas y contenedores.



## 1) Configuración de rutas y verificación de archivos
En esta celda definimos las rutas de entrada (los CSV) y de salida (carpetas para gráficas y modelo).  
También verificamos que existan los archivos necesarios.


In [ ]:

from pathlib import Path

# Directorios base (puedes ajustarlos si lo necesitas)
BASE_DIR = Path.cwd()                 # Directorio actual del notebook
PUBLIC_DIR = BASE_DIR / "public"      # Carpeta donde se esperan los CSV y se guardan salidas
DATA_RUTAS = PUBLIC_DIR / "bitacora_recoleccion_etl.csv"
DATA_CONTENEDOR = PUBLIC_DIR / "bitacora_contenedor_etl.csv"

# Directorios de salida
GRAF_DIR = PUBLIC_DIR / "graficas"
MODEL_DIR = PUBLIC_DIR / "modelos"
GRAF_DIR.mkdir(parents=True, exist_ok=True)   # Crea si no existen
MODEL_DIR.mkdir(parents=True, exist_ok=True)

print("Carpeta pública:", PUBLIC_DIR.resolve())
print("CSV de rutas:", DATA_RUTAS.resolve())
print("CSV de contenedor:", DATA_CONTENEDOR.resolve())

# Validación de existencia de archivos de datos
if not DATA_RUTAS.exists() or not DATA_CONTENEDOR.exists():
    raise FileNotFoundError(
        "No se encontraron los CSV necesarios. Coloca los archivos en la carpeta 'public':\n"
        f" - {DATA_RUTAS.name}\n"
        f" - {DATA_CONTENEDOR.name}"
    )



## 2) Importación de librerías
Aquí importamos las librerías para manejo de datos, modelado y evaluación.  
> **Nota:** Usamos **matplotlib** para los gráficos (sin seaborn) para mantener compatibilidad con entornos restringidos.


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Mostrar gráficos dentro del notebook
%matplotlib inline



## 3) Carga de datos
Leemos los dos CSV generados por tu proceso ETL:
- `bitacora_recoleccion_etl.csv`: información por **ruta** (ID, tiempo de duración, etc.).  
- `bitacora_contenedor_etl.csv`: información por **contenedor** relacionado a una bitácora/ruta (porcentaje de llenado, si fue recolectado, etc.).


In [ ]:

# Carga de datos limpios
df_rutas = pd.read_csv(DATA_RUTAS)
df_contenedor = pd.read_csv(DATA_CONTENEDOR)

print("df_rutas (shape):", df_rutas.shape)
print("df_contenedor (shape):", df_contenedor.shape)

# Vista rápida de las primeras filas para validar columnas
display(df_rutas.head())
display(df_contenedor.head())



## 4) Agregaciones por ruta (feature engineering)
A partir del detalle por contenedor, calculamos métricas **agregadas por ruta**:
- **Porcentaje_Recolectado**: promedio de `Recolectado` por `Bitacora_Id` (cuántos contenedores fueron recolectados en promedio).  
- **Promedio_Llenado**: promedio de `Porcentaje_Llenado` por `Bitacora_Id`.

Después, unimos estas métricas al `df_rutas` (tabla a nivel ruta).


In [ ]:

# Porcentaje de contenedores recolectados por ruta
recolectados = df_contenedor.groupby("Bitacora_Id")["Recolectado"].mean().reset_index()
recolectados = recolectados.rename(columns={"Recolectado": "Porcentaje_Recolectado"})

# Promedio de llenado por ruta
llenado = df_contenedor.groupby("Bitacora_Id")["Porcentaje_Llenado"].mean().reset_index()
llenado = llenado.rename(columns={"Porcentaje_Llenado": "Promedio_Llenado"})

# Unir agregados con la tabla de rutas (df_rutas)
df = df_rutas.merge(recolectados, left_on="ID", right_on="Bitacora_Id", how="left")
df = df.merge(llenado, left_on="ID", right_on="Bitacora_Id", how="left")

print("Dataset combinado (shape):", df.shape)
display(df.head())



## 5) Definir objetivo (`y`) y variables predictoras (`X`)
Regla de negocio para **Eficiente**:  
- `Tiempo_Duracion` ≤ **100** minutos **y**  
- `Porcentaje_Recolectado` ≥ **0.8** (80%)

Seleccionamos los **features** usados por el modelo y separamos en train/test.


In [ ]:

# Crear etiqueta objetivo (0 = no eficiente, 1 = eficiente)
df["Eficiente"] = ((df["Tiempo_Duracion"] <= 100) & (df["Porcentaje_Recolectado"] >= 0.8)).astype(int)

# Selección de variables predictoras (features)
features = ["Tiempo_Duracion", "Cantidad_Contenedores", "Promedio_Llenado", "Porcentaje_Recolectado"]
X = df[features].fillna(0)   # Reemplazamos posibles NaN por 0
y = df["Eficiente"]

# División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("Tamaños -> X_train:", X_train.shape, "| X_test:", X_test.shape)



## 6) Entrenamiento del modelo (Random Forest)
Entrenamos un **RandomForestClassifier** con 100 árboles y semilla fija para reproducibilidad.


In [ ]:

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Modelo entrenado.")



## 7) Guardado del modelo
Guardamos el modelo entrenado en `public/modelos/modelo_supervisado.pkl` usando `joblib`.


In [ ]:

import joblib

model_path = MODEL_DIR / "modelo_supervisado.pkl"
joblib.dump(clf, model_path)
print("Modelo guardado en:", model_path.resolve())



## 8) Visualizaciones
Creamos tres visualizaciones para entender el comportamiento del dataset y del modelo:
1. **Histograma** de `Tiempo_Duracion` por clase (`Eficiente` vs `No eficiente`).  
2. **Boxplot** de `Promedio_Llenado` por clase.  
3. **Matriz de confusión** del conjunto de prueba.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Asegurar carpetas de salida
GRAF_DIR.mkdir(parents=True, exist_ok=True)

# 8.1) Histograma de duración por clase
plt.figure(figsize=(6, 4))
plt.hist(df[df["Eficiente"] == 1]["Tiempo_Duracion"], bins=20, alpha=0.7, label="Eficiente")
plt.hist(df[df["Eficiente"] == 0]["Tiempo_Duracion"], bins=20, alpha=0.7, label="No eficiente")
plt.title("Duración de rutas por clase")
plt.xlabel("Tiempo_Duracion (min)")
plt.ylabel("Frecuencia")
plt.legend()
hist_path = GRAF_DIR / "hist_duracion.png"
plt.savefig(hist_path, bbox_inches="tight")
plt.show()
print("Guardado:", hist_path.resolve())

# 8.2) Boxplot de Promedio_Llenado por clase
plt.figure(figsize=(6, 4))
data0 = df[df["Eficiente"] == 0]["Promedio_Llenado"].dropna()
data1 = df[df["Eficiente"] == 1]["Promedio_Llenado"].dropna()
plt.boxplot([data0, data1], labels=["No eficiente", "Eficiente"])
plt.title("Promedio de llenado por clase")
plt.ylabel("Promedio_Llenado")
box_path = GRAF_DIR / "box_lleno.png"
plt.savefig(box_path, bbox_inches="tight")
plt.show()
print("Guardado:", box_path.resolve())

# 8.3) Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(4, 4))
plt.imshow(cm, interpolation="nearest", aspect="auto")
plt.title("Matriz de confusión")
plt.xlabel("Predicho")
plt.ylabel("Real")

# Etiquetas por celda
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha="center", va="center")

plt.colorbar()
conf_path = GRAF_DIR / "confusion.png"
plt.savefig(conf_path, bbox_inches="tight")
plt.show()
print("Guardado:", conf_path.resolve())



## 9) Métricas de evaluación
Calculamos **accuracy** y mostramos el **reporte de clasificación** (precision, recall, f1-score por clase).


In [ ]:

acc = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", round(acc, 4))
print("\nReporte de clasificación:\n", report)

resultado = {
    "accuracy": float(acc),
    "graficas": [str(hist_path), str(box_path), str(conf_path)]
}
resultado



## 10) (Opcional) Función de predicción para nuevos datos
Esta celda define una función `predecir` que recibe un `DataFrame` con las mismas columnas de `features` y devuelve la predicción del modelo.  
> Útil para validar casos manuales o nuevos registros.


In [ ]:

def predecir(df_nuevo):
    """
    Recibe un DataFrame con columnas:
    ['Tiempo_Duracion', 'Cantidad_Contenedores', 'Promedio_Llenado', 'Porcentaje_Recolectado']
    Devuelve un vector de predicciones (0 = No eficiente, 1 = Eficiente).
    """
    faltantes = [c for c in ['Tiempo_Duracion', 'Cantidad_Contenedores', 'Promedio_Llenado', 'Porcentaje_Recolectado'] if c not in df_nuevo.columns]
    if faltantes:
        raise ValueError(f"Faltan columnas requeridas: {faltantes}")
    X_new = df_nuevo[['Tiempo_Duracion', 'Cantidad_Contenedores', 'Promedio_Llenado', 'Porcentaje_Recolectado']].fillna(0)
    return clf.predict(X_new)

# Ejemplo de uso:
# import pandas as pd
# df_ejemplo = pd.DataFrame([
#     {'Tiempo_Duracion': 95, 'Cantidad_Contenedores': 15, 'Promedio_Llenado': 0.7, 'Porcentaje_Recolectado': 0.85},
#     {'Tiempo_Duracion': 120, 'Cantidad_Contenedores': 20, 'Promedio_Llenado': 0.6, 'Porcentaje_Recolectado': 0.75},
# ])
# predecir(df_ejemplo)



---

### ✅ Resumen
- **Datos de entrada:** `public/bitacora_recoleccion_etl.csv` y `public/bitacora_contenedor_etl.csv`  
- **Modelo:** RandomForestClassifier  
- **Salida:** `public/modelos/modelo_supervisado.pkl`  
- **Gráficas:** `public/graficas/`  
- **Métricas:** accuracy y reporte de clasificación

### 🔧 Sugerencias
- Si cambias la **regla de negocio** de `Eficiente`, edita la celda **5)**.
- Puedes ajustar hiperparámetros del RandomForest (e.g., `max_depth`, `min_samples_split`) para mejorar rendimiento.
